# Biosynthesis of undecylprodigiosin in Streptomyces coelicolor

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

#### Applications of the product

<div style="text-align: justify"> Undecylprodigiosin is a secondary metabolite belonging to the class of red-pigmented natural products known as the prodiginines, produced by a variety of bacteria, in particular Streptomyces spp, such as Streptomyces coelicolor and Streptomyces longisporus. The group is named after prodigiosin (prodiginine) and is biosynthesized through a common set of enzymes.  Bacterial prodiginines have a long history of being remarkable multipurpose compounds, best examined for their antibacterial and antimalarial activities, and they have received renewed attention because of potential clinical interests. Undecylprodigiosin can be used as an apoptotic and anticancer agent, showing antiproliferative effects in cancer cells.  In 2007, the selective apoptotic effect of undecylprodigiosin on breast carcinoma cells was found, and consequently, it was suggested “as a novel anticancer drug for the treatment of breast cancer”. Furthermore, recent research has shown potential applications of prodiginines in the food industry as coloring agents, antioxidants, and antimicrobial additives to increase product shelf life. Production of prodigiosin in the most common producer strain Serratia marcescens has been described in great detail. Particular emphasis is put on undecylprodigiosin which has generally been a less studied member of the prodigiosin family. </div>

#### Biosynthetic pathway

<div style="text-align: justify"> The tripyrrole molecules of prodigiosin and undecylprodigiosin are formed by the condensation of 4-methoxy-2,2'-bipyrrole-5-carbaldehyde (MBC) and either 2-methyl-3-n-amyl-pyrrole (MAP) for the prodigiosin production or 2-undecylpyrrole for the undecylprodigiocin production. The Serratia pig gene cluster and the Streptomyces red gene cluster responsible for these biosynthesis pathways have been identified. As can be seen in figure 3, the Streptomyces red gene cluster for undecylprodigiosin production includes the following genes: RedO, RedM, RedW, RedX, RedN, RedV, RedI, RedF, RedP, RedR, RedQ, RedL, RedK, RedH.  </div>

### 1.2 Literature review of the cell factory (<500 words)

####  Introduction to Streptomyces coelicolor

<div style="text-align: justify"> The cell factory utilized in this report is Streptomyces coelicolor, and its general advantages and disadvantages are discussed below.  </div>

#### General advantages

<div style="text-align: justify"> Streptomyces coelicolor A3(2) is amongst the best representatives of the genus Streptomyces, which is the largest genus of Actinobacteria. Streptomyces hasw a remarkably complex developmental life cycle and the capacity to produce a plethora of natural products such as anthelmintics, anticancer agents, and immunosuppressives. Today, 80% of the antibiotics are sourced from the genus Streptomyces, as the microorganisms belonging to this group have become the workhorse for genetics and a model for development and antibiotic production.  In particular, Streptomyces coelicolor are important bacteria and were sequenced because of their adaptability to environmental stress, source of bioactive molecules for medicine and industry, and relation to human pathogens. In parallel, many researchers have drawn attention to actinomycetes mainly because of their biosynthetic potential for the production of secondary metabolites, being one of the most prolific and important well-developed commercial products in the pharmaceutical, agrochemical, and food sectors. </div>

#### General disadvantages

<div style="text-align: justify"> Even though Streptomyces coelicolor is generally a good chassis for the production of several antibiotics, some issues remain to be addressed. One important drawback of Streptomyces fermentations is the need to optimize culture conditions in order to obtain the most optimal equilibrium between differentiation, cell death, and secondary metabolite production. Furthermore, some additional disadvantages of this strain include poor genetic tractability, a high level of production of endogenous metabolites, cross-regulation, and biochemical interactions between endogenous and heterologous gene clusters.  </div>

#### Suitability of the cell factory for the product

<div style="text-align: justify"> The suitability of the cell factory to produce undecylprodigiosin lies in the advantages described above, which include its capacity to produce a plethora of natural products and secondary metabolites, and the fact that Streptomyces coelicolor is a well-analyzed and well-known strain of the whole Streptomyces family. In Streptomyces coelicolor, undecylprodigiosin is biosynthesized encoding the red gene cluster, a cluster consisting of 23 genes. In the past few years, many researchers have helped in organizing and learning more about this cluster, and nowadays, it is available in the scientific community. An alternative organism for undecylprodigiosin production could be Serratia marcescens, using the pig gene cluster, however, it seems that the red gene cluster is better defined than the pig one, and as can be seen in Figure 4, there are some red genes in undecylprodigiosin production that cannot be replaced by the pig genes, as the latter are fewer than the former.  </div>

## 2. Problem definition (<300 words)

<div style="text-align: justify"> Even though the prodigionines family is well-understood, the undecylprodigiosin is less studied and there are some limitations to its industrial production by Streptomyces coelicolor.  Currently, undecylprodigiosin production mainly depends on chemical synthesis via microbial fermentation, a method that is economical and sustainable. However, in order for microbial production to be a successful platform that is economically viable, the fermentation process needs to result in adequate productivity, rates, and yields. Regarding the organism we use, Streptomyces coelicolor, although it is the best-studied undecylprodigiosin producer, it can only produce a low concentration of it. 

In this report, we want to examine ways to obtain overproduction of undecylprodigiosin in S. coelicolor in silico using the genome-scale metabolic model (GSM) Sco-GEM for the relevant strain M145. We will use different computational methods to identify metabolic changes that will result in increased undecylprodigiosin production. For this reason, we investigate different methods to achieve this, including media optimization, alternative carbon sources, upregulation and downregulation of specific genes, gene knockout, and co-facto swapping. </div>

## 3. Selection and assessment of existing GSM (<500 words)

<div style="text-align: justify"> Streptomyces coelicolor is an extensively used cell factory and model organism in basic biological research. Various industrial workhorse strains of S. coelicolor exist, due to recent development in systems biology and a strong interest in bacteria that can help in antibiotics production. Consequently, there are multiple GSMs available for S.coelicolor, which have undergone multiple rounds of curation and improvement since the first version was published. </div>

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project)

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References